# Writing a quantum program 2: bloq properties

Qualtran lets you write quantum programs and subroutines by composing lower-level subroutines, gates, and operations. We call these composable objects *bloqs* because they are the quantum building blocks of a complex algorithm. Composition is recursive: composing (lower-level) bloqs defines (higher-level) bloqs. 

In this tutorial, you will write your own bloq defined by a composition of subbloqs. You will query properties of the bloq by calling methods on the bloq object.

## Negating an integer, again

Recall from the first tutorial that we can negate a signed integer by performing a `BitwiseNot` and then adding the constant `k=1`.
We will start with the code from the prior tutorial. 

In [ ]:
# Quantum program 2
# Negate a quantum integer
# 
# Registers:
#   x: an 8-bit signed quantum integer

from qualtran import BloqBuilder
from qualtran import QInt

from qualtran.bloqs.arithmetic import BitwiseNot, AddK

# Set up input/output registers named 'x'
bb = BloqBuilder()
x = bb.add_register('x', QInt(8))

# Do the sub-operations
x = bb.add(BitwiseNot(QInt(8)), x=x)
x = bb.add(AddK(QInt(8), k=1), x=x)

# Finish up
negate = bb.finalize(x=x)

import qualtran.testing as qlt_testing
qlt_testing.assert_valid_cbloq(negate)

## Declaring a negation `Bloq`

Each re-usable subroutine is defined by a *bloq class* that inherits from the `qualtran.Bloq` abstract base class. While this introduces a degree of boilerplate not present in our original program, it lets you annotate your bloq with additional information that can later be used in analysis. 

We start by stubbing out a bloq class for our negation operation, which we name `Negate`. In this code snippet:

 - We declare a new Python class named `Negate` that implements the `qualtran.Bloq` interface.
 - Here and throughout the Qualtran standard library we use the [attrs](https://www.attrs.org/en/stable/) package to simplify the definition of Python classes. We recommend this for your own bloq definitions, but it isn't strictly required. In this snippet, we just get a few niceties (readible string representation, hashability, ...) from `attrs`. We'll describe any additional functionality later, as we use it.
 - We implement the one stricly-required method: `signature`. This method returns a specification that describes the input/output names and types of the bloq.

In [ ]:
import attrs
from qualtran import Bloq, Signature, Register

@attrs.frozen
class Negate(Bloq):
    """Negate a quantum integer.

    Registers:
        x: an 8-bit signed quantum integer
    """

    @property
    def signature(self) -> Signature:
        # The signature specifies input/output data types for the bloq.
        # It is a list of `Register` objects; each describing an input, output,
        # or input-output register.
        return Signature([
            # Our signature has one register named "x" with data type of QInt(8)
            Register('x', QInt(8)),
        ])

This is the minimum-viable bloq class. Note that we've just *declared* the quantum operation and its signature. Later we will *define* it by writing its decomposition into subbloqs. 

By instantiating the *bloq class* we can get a *bloq object*. These objects can be added to other quantum programs, and we can query their properties. Later, we'll see how a single bloq class can instantiate a variety of bloq objects by changing compile-time classical parameters; but for now, accept that you have to instantiate bloq classes into bloq objects.

In [ ]:
negate = Negate()

We can already query some properties of the bloq. Of course we can inspect its signature.

In [ ]:
print("Number of registers in `negate`:", len(negate.signature))

We can also estimate the number of qubits. Since we haven't provided an implementation for the bloq, this estimate will just use the signature and may be an underestimate.

In [ ]:
from qualtran.resource_counting import get_cost_value, QubitCount

print("Number of qubits in `negate` (underestimate):", get_cost_value(negate, QubitCount()))

## Implementing a negation `Bloq`

To implement the decomposition (i.e. circuit) for our negation subroutine, we need to port the code from the prior tutorial (reproduced at the top of this page) into a method on the bloq class. In particular, we implement the `build_composite_bloq` method. In the following snippet:

 - We copy the code from before. The set-up is the same.
 - We implement a method called `build_composite_bloq`. Unlike our free-form program, here the Qualtran framework will configure our `bb: BloqBuilder` object according to the bloq's signature; and input quantum variables (soquets) will be passed in&mdash;again according to the bloq's signature.

In [ ]:
import attrs
from qualtran import Bloq, Signature, Register

@attrs.frozen
class Negate(Bloq):
    """Negate a quantum integer.

    Registers:
        x: an 8-bit signed quantum integer
    """

    @property
    def signature(self) -> Signature:
        return Signature([
            Register('x', QInt(8)),
        ])
        
    def build_composite_bloq(self, bb: BloqBuilder, x):
        # The framework passes in a pre-configured BloqBuilder argument
        # additional parameters to this method should be named according
        # to the signature. For example, this method takes one additional
        # argument named "x" which corresponds to the input quantum variable "x".

        # The implementation of our subroutine wire up the subbloqs as before
        x = bb.add(BitwiseNot(QInt(8)), x=x)
        x = bb.add(AddK(QInt(8), k=1), x=x)

        # At the end, we associate our output regesters with the final
        # quantum variable.
        return {'x': x}

## Inspecting a negation bloq

Let's visualize our newly declared and defined bloq. First, we instantiate the bloq class into a bloq object.

In [ ]:
negate = Negate()

Let's draw the directed acyclic graph representation of the subroutine like we did in tutorial 1.

In [ ]:
from qualtran.drawing import show_bloq
show_bloq(negate)

Is that what you expected to see? The `show_bloq` function draws a single box for our `Negate` bloq. **Qualtran will always keep programs in a modular, heirarchical form unless specifically asked.** This is essential to support reasoning about large quantum programs with billions of gates.

We can get the expected implementation of the negation bloq by calling the `decompose_bloq` method. This will perform one level of decomposition; and is used extensivly during program analysis or visualization to recurse down the syntax tree.

In [ ]:
show_bloq(negate.decompose_bloq())

How do you compile your program down to individual one- and two- qubit atomic gates? Again, you **usually shouldn't do this**, and should instead design your analysis routine to work in a modular, recursive fashion. That being said: there is a method called `flatten` which does multiple in-place decompositions.

## Analyzing a negation bloq

We can compute various resource costs on the bloq.

In [ ]:
from qualtran.resource_counting import get_cost_value, QECGatesCost

get_cost_value(negate, QECGatesCost())

In [ ]:
get_cost_value(negate, QubitCount())

`show_call_graph` will show the bloq's *call graph*: the quantity and types of subbloqs involved in the decomposition. Here, we limit it to a finite depth. By default, the call graph will include some salient resource costs.

In [ ]:
from qualtran.drawing import show_call_graph
show_call_graph(negate, max_depth=2)

We can simulate certain classical-reversible gates (like this one) by using the `call_classically` method.

In [ ]:
negate.call_classically(x=5)

We can do a full statevector-like quantum simulation by requesting the bloq's *tensors*. This should only be attempted for bloqs with easy tensor contraction paths, like those with small qubit counts or particular amplitudes in shallow circuits.

In [ ]:
unitary = negate.tensor_contract()
unitary.shape

## Bloq classes define a family of bloq objects

Often times, we want to use compile-time classical parameters to change the construction of a quantum subroutine. Here, we modify our `Negate` example to define a *family* of bloq objects with different bitsizes. In this code snippet we:

 - Add a class attribute called `n` which stores the number of bits that this bloq will process.
 - Modify the signature of the bloq to use the parameter `n` to declare which data types we will process, namely signed integers of bitsize `n`.
 - Modify the subbloqs we call to again use the parameter `n`

In [ ]:
import attrs
from qualtran import Bloq, Signature, Register

@attrs.frozen
class Negate(Bloq):
    """Negate a quantum integer.

    Args:
        n: the number of bits

    Registers:
        x: an n-bit signed quantum integer
    """

    # the `attrs` decorator lets us define class attributes
    # with this simple name: type syntax.
    # This sets up one classical parameter "n", which should
    # be an integer.
    n: int

    @property
    def signature(self) -> Signature:
        return Signature([
            # Instead of hard-coding the value "8" for the number of bits,
            # we use the class attribute `n`.
            Register('x', QInt(bitsize=self.n)),
        ])
        
    def build_composite_bloq(self, bb: BloqBuilder, x):
        # Instead of hard-coding the value "8" for the number of bits
        # in the subbloqs, we use the class attribute `n`.
        x = bb.add(BitwiseNot(QInt(self.n)), x=x)
        x = bb.add(AddK(QInt(self.n), k=1), x=x)
        return {'x': x}

Throughout the standard library of bloqs, you will see many compile-time classical parameters making *generic* bloq classes that can generate bloq objects with different bitsizes and data types. The compile-time class attributes can also e.g. invert the control value of a controlled bloq. Finally, we support quantum metaprogramming by writing bloq classes that take a bloq object as a compile-time classical parameter. An example of this would be the phase estimation bloqs.

In [ ]:
# Now, we instantiate our bloq class into a bloq object
# by specifying the bitsize
negate = Negate(n=2048)
show_bloq(negate)

In [ ]:
# Even large registers can be supported performantly
show_bloq(negate.decompose_bloq())

## Symbolics

Sometimes, the bloq composition doesn't need a concrete value for its parameters. The `sympy` package is a Python library for doing symbolic computer algebra. Here and throughout the Qualtran standard library we can use `sympy.Symbol`s in place of concrete values.

In [ ]:
import sympy
n = sympy.Symbol('n')
negate = Negate(n=n)
show_bloq(negate.decompose_bloq())

Since we can do arithmetic on sympy symbols, many analysis protocols work on symbolic parameters including the resource costing features.

In [ ]:
get_cost_value(negate, QECGatesCost())